In [11]:
import random

import pickle
import matplotlib.pyplot as plt

import numpy as np 

from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
pass



In [12]:
#!pip install allennlp

In [13]:
#!pip install allennlp-models

In [14]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']

In [15]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [16]:
filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [17]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [18]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [19]:
action3 = ['work was unsatisfactory', 'performance was not up to the mark',
           'application was approved', 'application was not approved',
           'application was under review', 'work was appreciated', 
           'behaviour was unacceptable', 'project scope had been finalised']

verb_action[verb_list_p2[0]] = [action3]

In [20]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']
verb_action[verb_list_p2[1]] = [action4]

In [21]:
def predict_clusters(sentence):
    output = predictor.predict(document = sentence)
    return output['clusters'], output

In [22]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [23]:
with open('saved_pickles/Exploration/unique_input1_set.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set.pickle', 'rb') as handle:
    unique_input1_error_set = pickle.load(handle)

with open('saved_pickles/Exploration/occupation_pair_error.pickle', 'rb') as handle:
    occupation_pair_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_error.pickle', 'rb') as handle:
    occupation1_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_error.pickle', 'rb') as handle:
    occupation2_error = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_error.pickle', 'rb') as handle:
    verb_error = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_error.pickle', 'rb') as handle:
    action_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation_pair_count.pickle', 'rb') as handle:
    occupation_pair_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_count.pickle', 'rb') as handle:
    occupation1_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_count.pickle', 'rb') as handle:
    occupation2_count = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_count.pickle', 'rb') as handle:
    verb_count = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_count.pickle', 'rb') as handle:
    action_count = pickle.load(handle)

In [24]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [25]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [26]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict
        
        

In [27]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [28]:
# probability_dict = get_probability_dict(occupation_pair_error, occupation_pair_count)
# print(np.shape(list(probability_dict.keys())))

In [29]:
unique_input1_set_exploitation = set()
unique_input1_error_set_exploitation = set()

occupation_pair_error_exploitation = {}

occupation1_error_exploitation = {}

occupation2_error_exploitation = {}

verb_error_exploitation = {}

action_error_exploitation = {}

occupation_pair_count_exploitation = {}

occupation1_count_exploitation = {}

occupation2_count_exploitation = {}

verb_count_exploitation = {}

action_count_exploitation = {}


In [30]:
oc1_probability = get_probability_dict(occupation1_error, occupation1_count)

print(oc1_probability)
print()

error_rate_dict = get_error_rate_dict(occupation1_error, occupation1_count)
print(error_rate_dict)
print()

output_dict = {}
for i in range(100000):
    oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
    update_dict(output_dict, oc1)
print(get_sorted_dict(output_dict))

{'janitor': 0.0610288229667586, 'carpenter': 0.05967648037034397, 'guard': 0.058982651459974424, 'farmer': 0.05676955180180167, 'developer': 0.05631359168458105, 'mechanic': 0.0529287236894777, 'sheriff': 0.051313885007244056, 'driver': 0.0498247632962715, 'engineer': 0.04690012903919384, 'laborer': 0.046402108668284814, 'manager': 0.045852410408551696, 'physician': 0.04510720123980158, 'accountant': 0.04290710954889947, 'lawyer': 0.04174337554602117, 'construction worker': 0.035797543588386924, 'chief': 0.03476156938691793, 'cook': 0.03442990942918087, 'supervisor': 0.03417270913807923, 'analyst': 0.03307982413281862, 'salesperson': 0.03238549103250487, 'CEO': 0.02873835330403295, 'technician': 0.02675423852912795, 'mover': 0.024129556731744835}

{'janitor': 0.3789954337899543, 'carpenter': 0.37059724349157736, 'guard': 0.36628849270664504, 'farmer': 0.35254491017964074, 'developer': 0.3497133497133497, 'mechanic': 0.32869296210363497, 'sheriff': 0.3186646433990895, 'driver': 0.309417

In [ ]:
#TODO changes in exploration add

In [ ]:
#consider exploring the lowest probability parts too, like Inputs from HEll

In [ ]:
#particle swarm

In [33]:
def generate_test_sentences(ITERS=300): #3000):
    err_count = 0

    oc1_probability = get_probability_dict(occupation1_error, occupation1_count)
    oc2_probability = get_probability_dict(occupation2_error, occupation2_count)

    for i in range(ITERS):
#         oc1 = random.choice(occupations_1)
#         oc2 = random.choice(occupations_2)
        oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
        oc2 = get_weighted_random_choice(occupation2_error, occupation2_count, probablilities_dict=oc2_probability)
        verb = random.choice(list(verb_action.keys()))
        action = random.choice(random.choice(verb_action[verb]))
        pronoun = choose_pronoun_type(verb)
        input1 = ("The " + oc1 + " " + verb + " "
               + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 

        input2 = ("The " + oc1 + " " + verb + " "
               + oc2 + " " + filler_conjunction[verb] +  pronoun[1] + " " + action) 

        input3 = ("The " + oc1 + " " + verb + " "
               + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
        pred1, _ = predict_clusters(input1)
        pred2, _ = predict_clusters(input2)
        pred3, _ = predict_clusters(input3)


        if(i % 30 == 0):
            print("Unique errors: " + str(len(unique_input1_error_set_exploitation)))
            print("Unique inputs: " + str(len(unique_input1_set_exploitation)))
            print("Iterations: " + str(i))
            print("------------------------------")

        if input1 not in unique_input1_set:
            unique_input1_set_exploitation.add(input1)

        update_dict(occupation_pair_count_exploitation, (oc1, oc2))
        update_dict(occupation1_count_exploitation, oc1)
        update_dict(occupation2_count_exploitation, oc2)
        update_dict(verb_count_exploitation, verb)
        update_dict(action_count_exploitation, action)



        if not (pred1 == pred2 and pred2 == pred3):
            if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
                if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
    #         if(True):
                    err_count += 1
        
                    
                    if input1 not in unique_input1_error_set:
                        unique_input1_error_set_exploitation.add(input1)

    #                 print(pred1, pred2, pred3)
    #                 print(input1)
    #                 print(input2)
    #                 print(input3)

                    update_dict(occupation_pair_error_exploitation, (oc1, oc2))
                    update_dict(occupation1_error_exploitation, oc1)
                    update_dict(occupation2_error_exploitation, oc2)
                    update_dict(verb_error_exploitation, verb)
                    update_dict(action_error_exploitation, action)



    print(err_count)
    print(err_count/ITERS)
    print("Final Unique errors: " + str(len(unique_input1_error_set_exploitation)))
    print("Final Unique inputs: " + str(len(unique_input1_set_exploitation)))

In [34]:
generate_test_sentences()

Unique errors: 66
Unique inputs: 192
Iterations: 0
------------------------------
Unique errors: 66
Unique inputs: 203
Iterations: 30
------------------------------
Unique errors: 67
Unique inputs: 215
Iterations: 60
------------------------------
Unique errors: 68
Unique inputs: 229
Iterations: 90
------------------------------
Unique errors: 68
Unique inputs: 235
Iterations: 120
------------------------------
Unique errors: 69
Unique inputs: 251
Iterations: 150
------------------------------
Unique errors: 69
Unique inputs: 262
Iterations: 180
------------------------------
Unique errors: 69
Unique inputs: 271
Iterations: 210
------------------------------
Unique errors: 69
Unique inputs: 280
Iterations: 240
------------------------------
Unique errors: 69
Unique inputs: 292
Iterations: 270
------------------------------
8
0.02666666666666667
Final Unique errors: 69
Final Unique inputs: 304


In [77]:
# occupations_1 = ['supervisor',
# 'janitor',
# 'cook',
# 'mover',
# 'laborer',
# 'constructor',
# 'chief',
# 'developer',
# 'carpenter',
# 'manager',
# 'lawyer',
# 'farmer',
# 'driver',
# 'salesperson',
# 'physician',
# 'guard',
# 'analyst',
# 'mechanic',
# 'sheriff',
# 'CEO']

In [78]:
# occupations_2 = ['cashier',
# 'teacher',
# 'nurse',
# 'assistant',
# 'secretary',
# 'auditor',
# 'cleaner',
# 'receptionist',
# 'clerk',
# 'counselors',
# 'designer',
# 'hairdresser',
# 'attendant',
# 'writer',
# 'housekeeper',
# 'baker',
# 'accountant',
# 'editor',
# 'librarian',
# 'sewer']
# occupations_1.extend(["technician", "accountant", "engineer"])
# occupations_2.extend(["teacher", "librarian", "nurse", "paralegal"])

In [98]:
# verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
# verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [99]:
# error_rate_dict = get_error_rate_dict(occupation1_error, occupation1_count)
# for key in error_rate_dict:
#     print(key, error_rate_dict[key])

janitor 0.3789954337899543
carpenter 0.37059724349157736
guard 0.36628849270664504
farmer 0.35254491017964074
developer 0.3497133497133497
mechanic 0.32869296210363497
sheriff 0.3186646433990895
driver 0.3094170403587444
engineer 0.29125475285171104
laborer 0.2881619937694704
manager 0.284748309541698
physician 0.28012048192771083
accountant 0.2664576802507837
lawyer 0.2592307692307692
construction worker 0.22230652503793627
chief 0.21587301587301588
cook 0.21381337252020574
supervisor 0.21221613155833985
analyst 0.20542920029347028
salesperson 0.2011173184357542
CEO 0.17846820809248554
technician 0.16614664586583464
mover 0.14984709480122324


In [100]:
# error_rate_dict = get_error_rate_dict(occupation2_error, occupation2_count)
# for key in error_rate_dict:
#     print(key, error_rate_dict[key])

housekeeper 0.7448979591836735
teacher 0.5567375886524822
nurse 0.5499583680266444
paralegal 0.5441640378548895
assistant 0.4641449960598897
cleaner 0.3044189852700491
hairdresser 0.28137817883511074
receptionist 0.23923444976076555
counselor 0.20754716981132076
librarian 0.18018377946464242
writer 0.16612377850162866
baker 0.16346153846153846
cashier 0.13739266198282593
secretary 0.12580645161290321
attendant 0.10501567398119123
clerk 0.10364372469635627
accountant 0.07798537774167344
designer 0.07322834645669292
tailor 0.07307386814932486
editor 0.05304829770387965
auditor 0.036057692307692304


In [81]:
# error_rate_dict = get_error_rate_dict(occupation_pair_error, occupation_pair_count)
# for key in error_rate_dict:
#     print(key, error_rate_dict[key], occupation_pair_count[key])

('driver', 'teacher') 0.9363636363636364 110
('mechanic', 'teacher') 0.9230769230769231 117
('farmer', 'housekeeper') 0.9166666666666666 48
('janitor', 'housekeeper') 0.9076923076923077 65
('carpenter', 'teacher') 0.896 125
('developer', 'housekeeper') 0.8867924528301887 53
('laborer', 'housekeeper') 0.8813559322033898 59
('guard', 'nurse') 0.8761904761904762 105
('mechanic', 'housekeeper') 0.8656716417910447 67
('engineer', 'teacher') 0.8620689655172413 116
('sheriff', 'housekeeper') 0.86 50
('laborer', 'teacher') 0.8543689320388349 103
('manager', 'housekeeper') 0.8478260869565217 46
('lawyer', 'housekeeper') 0.8392857142857143 56
('janitor', 'nurse') 0.8314606741573034 89
('driver', 'housekeeper') 0.8307692307692308 65
('lawyer', 'paralegal') 0.8297872340425532 47
('physician', 'housekeeper') 0.8269230769230769 52
('farmer', 'teacher') 0.8203125 128
('physician', 'nurse') 0.8198198198198198 111
('farmer', 'nurse') 0.8058252427184466 103
('construction worker', 'teacher') 0.793814432

In [85]:
# print(len(occupation_pair_count.keys()))
# print(len(occupation1_count.keys()) * len(occupation2_count.keys()) )

483
483


def generate(